In [1]:
import os
import json
import tensorflow as tf
import numpy as np
from pycocotools.cocoeval import COCOeval

from detection.datasets import coco, data_generator
from detection.models.detectors import faster_rcnn
from tensorflow.python.keras.layers.normalization import BatchNormalization

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
img_mean = (123.675, 116.28, 103.53)
# img_std = (58.395, 57.12, 57.375)
img_std = (1., 1., 1.)
batch_size = 8
val_dataset = coco.CocoDataSet('/workspace/shared_workspace/data/coco/', 'val',
                               flip_ratio=0,
                               pad_mode='fixed',
                               mean=img_mean,
                               std=img_std,
                               scale=(800, 1216))

val_generator = data_generator.DataGenerator(val_dataset, shuffle=True)
val_tf_dataset = tf.data.Dataset.from_generator(
    val_generator, (tf.float32, tf.float32, tf.float32, tf.int32))

val_tf_dataset = val_tf_dataset.prefetch(256) #.shuffle(100).shard(hvd.size(), hvd.rank())
val_tf_dataset = val_tf_dataset.padded_batch(
                            batch_size,
                            padded_shapes=(
                            tf.TensorShape([None, None, 3]), # image padded to largest in batch
                            tf.TensorShape([11]),            # image meta - no padding
                            tf.TensorShape([None, 4]),       # bounding boxes, padded to longest
                            tf.TensorShape([None])           # labels, padded to longest
                            ),
                            padding_values=(0.0, 0.0, 0.0, -1))

# flip channel order
def flip_channels(img, img_meta, bbox, label):
    img = tf.reverse(img, axis=[-1])
    return img, img_meta, bbox, label

val_tf_dataset = val_tf_dataset.filter(lambda w, x, y, z: tf.equal(tf.shape(w)[0], batch_size))
val_tf_dataset = val_tf_dataset.map(flip_channels, num_parallel_calls=16)
val_tf_dataset = iter(val_tf_dataset.repeat())


loading annotations into memory...
Done (t=0.60s)
creating index...
index created!


In [4]:
model = faster_rcnn.FasterRCNN(
    num_classes=len(val_dataset.get_categories()), batch_size=batch_size)

for i in val_tf_dataset:
    _ = model(i)
    break

In [5]:
model.layers[0].trainable=False

for layer in model.layers[0].layers[0].layers[80:]:
    if type(layer)!=BatchNormalization:
        layer.trainable=True

In [6]:
model.load_weights( \
'/workspace/shared_workspace/tf-eager-fasterrcnn/rcnn_keras_resnet_50_stage_3.h5')

In [7]:
imgs, img_metas, bboxes, labels = next(val_tf_dataset)

In [8]:
@tf.function(experimental_relax_shapes=True)
def evaluate(imgs, img_metas):
    c2, c3, c4, c5 = model.backbone(imgs, training=False)
    p2, p3, p4, p5, p6 = model.neck((c2, c3, c4, c5), training=False)
    rpn_class_logits, rpn_probs, rpn_deltas = model.rpn_head((p2, p3, p4, p5, p6), training=False)
    proposals = model.rpn_head.get_proposals(rpn_probs, rpn_deltas, img_metas)
    pooled_regions = model.roi_align((proposals, (p2, p3, p4, p5), img_metas), training=False)
    cnn_class_logits, rcnn_probs, rcnn_deltas = \
                model.bbox_head(pooled_regions, training=False)
    detections_list = model.bbox_head.get_bboxes(
                    rcnn_probs, rcnn_deltas, proposals, img_metas)
    return detections_list

In [9]:
detection = evaluate(imgs, img_metas)

OperatorNotAllowedInGraphError: in converted code:

    <ipython-input-8-f820098572bb>:10 evaluate  *
        detections_list = model.bbox_head.get_bboxes(
    /workspace/shared_workspace/tf-eager-fasterrcnn/detection/models/bbox_heads/bbox_head.py:110 get_bboxes  *
        detections_list = [
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/ops.py:539 __iter__
        self._disallow_iteration()
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/ops.py:532 _disallow_iteration
        self._disallow_when_autograph_enabled("iterating over `tf.Tensor`")
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/ops.py:510 _disallow_when_autograph_enabled
        " decorating it directly with @tf.function.".format(task))

    OperatorNotAllowedInGraphError: iterating over `tf.Tensor` is not allowed: AutoGraph did not convert this function. Try decorating it directly with @tf.function.
